In [1]:
import os

In [2]:
! git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [3]:
!cp "Pneumonia.yaml" "yolov5/"
!cp "yolov5sp.yaml" "yolov5/models/"
os.chdir("yolov5")
!rm -rf pneumonia_data
!mkdir pneumonia_data
os.chdir("pneumonia_data")
!mkdir images
!mkdir labels
os.chdir("images")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir("labels")
!mkdir train
!mkdir validation
!mkdir test
os.chdir(os.pardir)
os.chdir(os.pardir)

In [4]:
import pandas as pd
import os
import ast
import shutil
from sklearn import model_selection
from tqdm import tqdm
import numpy as np
!pip install -r requirements.txt

In [5]:
pip install -U PyYAML

Requirement already up-to-date: PyYAML in /home/bhavya/my_project_dir/my_project_env/lib/python3.8/site-packages (5.3.1)
Note: you may need to restart the kernel to use updated packages.


In [6]:
DATA_PATH = "rsna-pneumonia-detection-challenge"
OUTPUT_PATH = "yolov5/pneumonia_data"

In [7]:
def process_data(data,data_type="train"):
    for _,row in tqdm(data.iterrows(),total=len(data),position=0,leave=True):
        image_name = row["patientId"]
        bounding_boxes = row["bboxes"]
        yolo_data = []
        for bbox in bounding_boxes:
            if(bbox[4]=="0.0"):
              # print("hello")
              continue
            x=float(bbox[0])
            y=float(bbox[1])
            w=float(bbox[2])
            h=float(bbox[3])
            x_center = x+w/2
            y_center = y+h/2
            x_center/=1024.0
            y_center/=1024.0
            w/=1024.0
            h/=1024.0
            yolo_data.append([0,x_center,y_center,w,h])
        yolo_data = np.array(yolo_data)
        np.savetxt(
            os.path.join(OUTPUT_PATH,f"labels/{data_type}/{image_name}.txt"),
            yolo_data,
            fmt = ["%d","%f","%f","%f","%f"]
        )
        shutil.copyfile(
            os.path.join(DATA_PATH,f"train/{image_name}.jpg"),
            os.path.join(OUTPUT_PATH,f"images/{data_type}/{image_name}.jpg")
        )

In [8]:
os.chdir(os.pardir)

In [9]:
os.getcwd()

'/home/bhavya/Downloads/Pneumonia-detection'

In [11]:
if __name__=="__main__":
    df = pd.read_csv(os.path.join(DATA_PATH,"train.csv"))
#     df_test = df_full.iloc[int(0.9*len(df_full)):]
    df = df[df['Target']==1]
    # df.bbox = df.bbox.apply(ast.literal_eval)
    cols = ['x', 'y', 'width','height','Target']
    df['combined'] = df[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)
    df['combined'] = df.combined.apply(lambda x: x.strip('()').split(','))
    # print(type(df['combined'].iloc[0]))
    # print(df)
    df = df.groupby("patientId")['combined'].apply(list).reset_index(name="bboxes")
    # print(df)
    df_train,df_valid = model_selection.train_test_split(
        df,
        test_size=0.11,
        random_state=True,
        shuffle=True,
    )
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    # print(df_train)
    process_data(df_train,data_type="train")
    process_data(df_valid,data_type="validation")

100%|██████████| 662/662 [00:01<00:00, 347.42it/s]


In [16]:
os.chdir("yolov5")
os.getcwd()

'/home/bhavya/Downloads/Pneumonia-detection/yolov5'

In [19]:
!python3 train.py --img 1024 --batch 8 --epochs 20 --data Pneumonia.yaml --cfg models/yolov5sp.yaml --name wm --weights pre_trained.pt --single-cls


Using torch 1.7.0 CUDA:0 (GeForce 940MX, 2004MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=False, cfg='models/yolov5sp.yaml', data='Pneumonia.yaml', device='', epochs=20, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[1024, 1024], local_rank=-1, log_imgs=16, multi_scale=False, name='wm', noautoanchor=False, nosave=False, notest=False, project='runs/train', rect=False, resume=False, save_dir='runs/train/wm3', single_cls=True, sync_bn=False, total_batch_size=8, weights='pre_trained.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2020-11-19 15:06:12.577610: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-11-19 15:06:12.577653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [20]:
!python detect.py --source "rsna-pneumonia-detection-challenge/test"  --weights "/yolov5/runs/exp0/weights/last.pt" --save-txt --conf 0.4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='rsna-pneumonia-detection-challenge/test', update=False, view_img=False, weights=['/yolov5/runs/exp0/weights/last.pt'])
Using torch 1.7.0 CUDA:0 (GeForce 940MX, 2004MB)

Traceback (most recent call last):
  File "detect.py", line 172, in <module>
    detect()
  File "detect.py", line 33, in detect
    model = attempt_load(weights, map_location=device)  # load FP32 model
  File "/home/bhavya/Downloads/Pneumonia-detection/yolov5/models/experimental.py", line 137, in attempt_load
    model.append(torch.load(w, map_location=map_location)['model'].float().fuse().eval())  # load FP32 model
  File "/home/bhavya/my_project_dir/my_project_env/lib/python3.8/site-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/home/bhavya/my_project